# Đọc hiểu file excel
## File portfolio_raw.xlsx chứa:

- Ngay: Ngày giao dịch
- MaCK: Mã chứng khoán (VNM, VCB, HPG, VHM, FPT, MSN, VIC, GAS, SAB, PLX)
- GiaMoCua: Giá mở cửa
- GiaCaoNhat: Giá cao nhất trong ngày
- GiaThapNhat: Giá thấp nhất trong ngày
- GiaDongCua: Giá đóng cửa
- KhoiLuongGD: Khối lượng giao dịch (cổ phiếu)
- GiaTriGD: Giá trị giao dịch (VNĐ)
- SoLuongNamGiu: Số CP quỹ đang nắm giữ

# Import Library

In [101]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Phần 1: Làm sạch dữ liệu (15 điểm)

## 1. Xử lý cơ bản (4 điểm)

- Ngày tháng → datetime, xử lý invalid
- MaCK → uppercase, trim
- Loại bỏ trùng lặp (Ngay + MaCK)

In [102]:
path = 'portfolio_raw.xlsx'
data = pd.read_excel(path)
data.head(10)

,Ngay,MaCK,GiaMoCua,GiaCaoNhat,GiaThapNhat,GiaDongCua,KhoiLuongGD,GiaTriGD,SoLuongNamGiu
0,2024-10-02,HPG,26000.0,27300.0,24700.0,24700.0,8604246.0,2.127110e+11,100000.0
1,2024-03-20,FPT,117900.0,122700.0,113100.0,116600.0,862699.0,1.005988e+11,25000.0
2,2024-08-22,MSN,91100.0,93500.0,88700.0,93200.0,1503009.0,1.400738e+11,35000.0
3,2024-08-12,MSN,91800.0,93600.0,90100.0,92600.0,5211975.0,4.828693e+11,35000.0
4,2024-02-22,HPG,27100.0,27800.0,26400.0,27000.0,648951.0,1.753332e+10,100000.0
5,2024-01-09,GAS,120300.0,123400.0,117200.0,118000.0,2361643.0,2.786530e+11,20000.0
6,2024-09-26,HPG,26600.0,27400.0,25700.0,26900.0,8945040.0,2.406605e+11,100000.0
7,2024-05-13,HPG,22800.0,23200.0,22400.0,1000000.0,5441534.0,1.258675e+11,100000.0
8,2024-10-14,SAB,NaN,155300.0,148000.0,151100.0,3154686.0,4.765544e+11,15000.0
9,2024-05-05,HPG,22800.0,23500.0,22100.0,22600.0,9023617.0,2.042053e+11,100000.0


In [103]:
data.shape

(3733, 9)

In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3733 entries, 0 to 3732
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ngay           3713 non-null   object 
 1   MaCK           3733 non-null   object 
 2   GiaMoCua       3712 non-null   float64
 3   GiaCaoNhat     3712 non-null   float64
 4   GiaThapNhat    3712 non-null   float64
 5   GiaDongCua     3718 non-null   float64
 6   KhoiLuongGD    3723 non-null   float64
 7   GiaTriGD       3719 non-null   float64
 8   SoLuongNamGiu  3706 non-null   float64
dtypes: float64(7), object(2)
memory usage: 262.6+ KB


In [105]:
columns = data.columns
for col in columns:
    print(f"Ty le Null cot {col}: {data[col].isnull().sum() / len(data[col]):.4f}%")

Ty le Null cot Ngay: 0.0054%
Ty le Null cot MaCK: 0.0000%
Ty le Null cot GiaMoCua: 0.0056%
Ty le Null cot GiaCaoNhat: 0.0056%
Ty le Null cot GiaThapNhat: 0.0056%
Ty le Null cot GiaDongCua: 0.0040%
Ty le Null cot KhoiLuongGD: 0.0027%
Ty le Null cot GiaTriGD: 0.0038%
Ty le Null cot SoLuongNamGiu: 0.0072%


In [106]:
#Chuan hoa cot Ngay
data['Ngay'] = pd.to_datetime(data['Ngay'], errors='coerce')
data['Ngay'] = data['Ngay'].ffill()
data['Ngay'].isnull().sum()

np.int64(0)

In [107]:
#Chuan hoa cot MCK
data['MaCK'] = data['MaCK'].str.upper().str.strip()
data['MaCK'].isnull().sum()

np.int64(0)

In [108]:
data = data.drop_duplicates(subset=['Ngay', 'MaCK'], keep = 'first')
data.shape

(3591, 9)

## 2. Xử lý giá OHLC (6 điểm)
- Kiểm tra logic: GiaCaoNhat ≥ GiaMoCua, GiaDongCua, GiaThapNhat
- GiaThapNhat ≤ GiaMoCua, GiaDongCua, GiaCaoNhat
- Nếu GiaCaoNhat < GiaThapNhat → hoán đổi
- Giá null → điền bằng median theo MaCK

In [109]:
df = data.copy()
def check_gia_logic(row, data):
    """
    - Kiểm tra logic: GiaCaoNhat ≥ GiaMoCua, GiaDongCua, GiaThapNhat
    - GiaThapNhat ≤ GiaMoCua, GiaDongCua, GiaCaoNhat
    - Nếu GiaCaoNhat < GiaThapNhat → hoán đổi
    - Giá null → điền bằng median theo MaCK
    - Giá âm hoặc > 500,000 → điền lại ( cho median theo MaCk)
    """
    mack_median = data.groupby('MaCK')['GiaDongCua'].median().loc[row['MaCK']]
    gia_mo_cua = row['GiaMoCua']
    gia_dong_cua = row['GiaDongCua']
    gia_cao_nhat = row['GiaCaoNhat']
    gia_thap_nhat = row['GiaThapNhat']
    if pd.isnull(gia_mo_cua):
        gia_mo_cua = mack_median
    if pd.isnull(gia_dong_cua):
        gia_dong_cua = mack_median
    if pd.isnull(gia_cao_nhat):
        gia_cao_nhat = mack_median
    if pd.isnull(gia_thap_nhat):
        gia_thap_nhat = mack_median
    if gia_cao_nhat < gia_thap_nhat:
        row['GiaCaoNhat'], row['GiaThapNhat'] = gia_thap_nhat, gia_cao_nhat
    if gia_mo_cua < 0 or gia_mo_cua > 500000:
        gia_mo_cua = mack_median
    if gia_dong_cua < 0 or gia_dong_cua > 500000:
        gia_dong_cua = mack_median
    if gia_cao_nhat < 0 or gia_cao_nhat > 500000:
        gia_cao_nhat = mack_median
    if gia_thap_nhat < 0 or gia_thap_nhat > 500000:
        gia_thap_nhat = mack_median
    gia_cao_nhat = max(gia_cao_nhat, gia_mo_cua, gia_dong_cua, gia_thap_nhat)
    gia_thap_nhat = min(gia_thap_nhat, gia_mo_cua, gia_dong_cua, gia_cao_nhat)
    
    return pd.Series([gia_mo_cua, gia_dong_cua, gia_cao_nhat, gia_thap_nhat])
df[['GiaMoCua', 'GiaDongCua', 'GiaCaoNhat', 'GiaThapNhat']] = df.apply(check_gia_logic, data = df,axis=1)

In [110]:
print(df['GiaMoCua'].isnull().sum(), df['GiaDongCua'].isnull().sum(), df['GiaCaoNhat'].isnull().sum(), df['GiaThapNhat'].isnull().sum())

0 0 0 0


## 3. Xử lý khối lượng và giá trị (3 điểm)

- KhoiLuongGD: hợp lệ 1,000 - 50,000,000 CP
- GiaTriGD phải = GiaDongCua × KhoiLuongGD
- Nếu sai → tính lại GiaTriGD

In [111]:
df1 = df.copy()
def khoi_luong_gd(row):
    """
    - KhoiLuongGD: hợp lệ 1,000 - 50,000,000 CP => dien laij theo median dua tren MaCK
    - GiaTriGD phải = GiaDongCua × KhoiLuongGD
    - Nếu sai → tính lại GiaTriGD
    """
    median_mack = df.groupby('MaCK')['KhoiLuongGD'].median().loc[row['MaCK']]
    if row['KhoiLuongGD'] < 1000 or row['KhoiLuongGD'] > 50000000:
        row['KhoiLuongGD'] = median_mack
    tmp = row['GiaDongCua'] * row['KhoiLuongGD']
    if row['GiaTriGD'] != tmp:
        row['GiaTriGD'] = tmp
    return pd.Series([row['KhoiLuongGD'], row['GiaTriGD']])

df1[['KhoiLuongGD', 'GiaTriGD']] = df1.apply(khoi_luong_gd, axis=1)

## 4. Xử lý số lượng nắm giữ (2 điểm)
- SoLuongNamGiu ≥ 0
- Null → điền 0

In [112]:
df2 = df1.copy()
def so_luong_nam_giu(row):
    """
    - SoLuongNamGiu ≥ 0
    -neu < 0 => dien so luong trung binh dua tren MaCK
    - Null → điền 0
    """
    tmp = row['SoLuongNamGiu']
    mean_mack = df2.groupby('MaCK')['SoLuongNamGiu'].mean().loc[row['MaCK']]
    if pd.isnull(tmp):
        tmp = 0
    if tmp < 0:
        tmp = mean_mack
    return tmp
df2['SoLuongNamGiu'] = df2.apply(so_luong_nam_giu, axis=1)

In [113]:
import os
if not os.path.exists('output'):
    os.makedirs('output')
df2.to_excel('output/portfolio_cleaned.xlsx', index = False)
df2.to_csv('output/portfolio_cleaned.csv', index = False)

# Phần 2: Tính toán chỉ số kỹ thuật (10 điểm)

## 5. Chỉ số cơ bản (3 điểm)

- BienDongNgay = GiaDongCua - GiaMoCua
- TyLeBienDong (%)
- AmplitudeBienDong = GiaCaoNhat - GiaThapNhat
- BienDongSoVoiHomTruoc (% so với ngày trước)

In [114]:
df3 = df2.copy()
df3['BienDongNgay'] = df3['GiaDongCua'] - df3['GiaMoCua']
df3['TyLeBienDong'] = (df3['BienDongNgay'] / df3['GiaMoCua'] * 100).round(2).astype(str) + '%'
df3['AmplitudeBienDong'] = df3['GiaCaoNhat'] - df3['GiaThapNhat']
# Group by 'MaCK' và tính BienDongSoVoiHomTruoc cho từng nhóm
df3['BienDongSoVoiHomTruoc'] = (df3.groupby('MaCK')['GiaDongCua'].transform(
    lambda group: (group - group.shift(1)) / group.shift(1) * 100
)).round(2).astype(str) + '%'
def fix_infinity(value):
    if value == 'nan%' or value == 'inf%' or value == '-inf%':
        return '0.0%'
    return value
df3['BienDongSoVoiHomTruoc'] = df3['BienDongSoVoiHomTruoc'].apply(fix_infinity)  
df3.head(10)

,Ngay,MaCK,GiaMoCua,GiaCaoNhat,GiaThapNhat,GiaDongCua,KhoiLuongGD,GiaTriGD,SoLuongNamGiu,BienDongNgay,TyLeBienDong,AmplitudeBienDong,BienDongSoVoiHomTruoc
0,2024-10-02,HPG,26000.0,27300.0,24700.0,24700.0,8604246.0,2.125249e+11,100000.0,-1300.0,-5.0%,2600.0,0.0%
1,2024-03-20,FPT,117900.0,122700.0,113100.0,116600.0,862699.0,1.005907e+11,25000.0,-1300.0,-1.1%,9600.0,0.0%
2,2024-08-22,MSN,91100.0,93500.0,88700.0,93200.0,1503009.0,1.400804e+11,35000.0,2100.0,2.31%,4800.0,0.0%
3,2024-08-12,MSN,91800.0,93600.0,90100.0,92600.0,5211975.0,4.826289e+11,35000.0,800.0,0.87%,3500.0,-0.64%
4,2024-02-22,HPG,27100.0,27800.0,26400.0,27000.0,648951.0,1.752168e+10,100000.0,-100.0,-0.37%,1400.0,9.31%
5,2024-01-09,GAS,120300.0,123400.0,117200.0,118000.0,2361643.0,2.786739e+11,20000.0,-2300.0,-1.91%,6200.0,0.0%
6,2024-09-26,HPG,26600.0,27400.0,25700.0,26900.0,8945040.0,2.406216e+11,100000.0,300.0,1.13%,1700.0,-0.37%
7,2024-05-13,HPG,22800.0,25000.0,22400.0,25000.0,5441534.0,1.360384e+11,100000.0,2200.0,9.65%,2600.0,-7.06%
8,2024-10-14,SAB,149200.0,155300.0,148000.0,151100.0,3154686.0,4.766731e+11,15000.0,1900.0,1.27%,7300.0,0.0%
9,2024-05-05,HPG,22800.0,23500.0,22100.0,22600.0,9023617.0,2.039337e+11,100000.0,-200.0,-0.88%,1400.0,-9.6%


## 6. Moving Averages (3 điểm)
- MA5: Trung bình động 5 ngày
- MA20: Trung bình động 20 ngày
- MA50: Trung bình động 50 ngày

In [115]:
df3 = df2.copy()
# Tính MA5/MA20/MA50 theo từng MaCK (không rolling trên toàn bộ DF)
df3 = df3.sort_values(['MaCK', 'Ngay'])
df3['MA5'] = df3.groupby('MaCK')['GiaDongCua'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())
df3['MA20'] = df3.groupby('MaCK')['GiaDongCua'].transform(lambda x: x.rolling(window=20, min_periods=1).mean())
df3['MA50'] = df3.groupby('MaCK')['GiaDongCua'].transform(lambda x: x.rolling(window=50, min_periods=1).mean())
print(df3[['MaCK','Ngay','GiaDongCua', 'MA5', 'MA20', 'MA50']].head(30))  # kiểm tra

     MaCK       Ngay  GiaDongCua       MA5           MA20           MA50
2737  FPT 2024-01-01    107300.0  107300.0  107300.000000  107300.000000
2978  FPT 2024-01-02    113100.0  110200.0  110200.000000  110200.000000
990   FPT 2024-01-03    109600.0  110000.0  110000.000000  110000.000000
555   FPT 2024-01-04    108600.0  109650.0  109650.000000  109650.000000
152   FPT 2024-01-05    110400.0  109800.0  109800.000000  109800.000000
3526  FPT 2024-01-06    109000.0  110140.0  109666.666667  109666.666667
3413  FPT 2024-01-07    112000.0  109920.0  110000.000000  110000.000000
2722  FPT 2024-01-08    109700.0  109940.0  109962.500000  109962.500000
1223  FPT 2024-01-09    115500.0  111320.0  110577.777778  110577.777778
2166  FPT 2024-01-10    116100.0  112460.0  111130.000000  111130.000000
3196  FPT 2024-01-11    117900.0  114240.0  111745.454545  111745.454545
87    FPT 2024-01-12    110400.0  113920.0  111633.333333  111633.333333
2200  FPT 2024-01-13    114900.0  114960.0  111884.

In [116]:
df3.head(10)

,Ngay,MaCK,GiaMoCua,GiaCaoNhat,GiaThapNhat,GiaDongCua,KhoiLuongGD,GiaTriGD,SoLuongNamGiu,MA5,MA20,MA50
2737,2024-01-01,FPT,109600.0,113100.0,106000.0,107300.0,4723048.0,5.067831e+11,25000.0,107300.0,107300.000000,107300.000000
2978,2024-01-02,FPT,111000.0,116400.0,105600.0,113100.0,7924409.0,8.962507e+11,25000.0,110200.0,110200.000000,110200.000000
990,2024-01-03,FPT,107900.0,109600.0,106200.0,109600.0,7297937.0,7.998539e+11,25000.0,110000.0,110000.000000,110000.000000
555,2024-01-04,FPT,109500.0,110800.0,108200.0,108600.0,5968701.0,6.482009e+11,25000.0,109650.0,109650.000000,109650.000000
152,2024-01-05,FPT,109600.0,112800.0,106500.0,110400.0,8055860.0,8.893669e+11,25000.0,109800.0,109800.000000,109800.000000
3526,2024-01-06,FPT,109500.0,112200.0,106900.0,109000.0,9367255.0,1.021031e+12,25000.0,110140.0,109666.666667,109666.666667
3413,2024-01-07,FPT,111600.0,116600.0,106600.0,112000.0,1049663.0,1.175623e+11,25000.0,109920.0,110000.000000,110000.000000
2722,2024-01-08,FPT,110200.0,111700.0,108700.0,109700.0,9300264.0,1.020239e+12,25000.0,109940.0,109962.500000,109962.500000
1223,2024-01-09,FPT,111800.0,117100.0,106500.0,115500.0,6308395.0,7.286196e+11,25000.0,111320.0,110577.777778,110577.777778
2166,2024-01-10,FPT,111900.0,116600.0,107200.0,116100.0,7402161.0,8.593909e+11,25000.0,112460.0,111130.000000,111130.000000


## 7. RSI (Relative Strength Index) (2 điểm)
- Tính RSI(14) cho mỗi mã CK
- Phân loại: Quá mua (>70), Quá bán (<30), Bình thường

In [117]:
def cal_rsi_14(data, period):
    """
    Tính RSI(14) cho mỗi MaCK, giữ nguyên index ban đầu để gán lại an toàn.
    Trả về dataframe có cột 'RSI_14' và 'RSI_Label'.
    """
    df = data.copy()
    df = df.sort_values(by=['MaCK', 'Ngay'])

    def _rsi(group):
        delta = group['GiaDongCua'].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(com=period - 1, adjust=False).mean()
        avg_loss = loss.ewm(com=period - 1, adjust=False).mean()
        rs = avg_gain / avg_loss
        rs = rs.replace([np.inf, -np.inf], np.nan)
        rsi = 100 - (100 / (1 + rs))
        return rsi

    # groupby.apply trả về Series với MultiIndex (MaCK, original_index), nên drop level MaCK để khớp index gốc
    rsi_series = df.groupby('MaCK').apply(_rsi)
    if isinstance(rsi_series.index, pd.MultiIndex):
        rsi_series.index = rsi_series.index.droplevel(0)

    # Gán lại vào dataframe sao cho index khớp
    df.loc[rsi_series.index, 'RSI_14'] = rsi_series

    # Phân loại RSI
    def classify(x):
        if pd.isna(x):
            return 'Bình thường'
        if x > 70:
            return 'Quá mua'
        if x < 30:
            return 'Quá bán'
        return 'Bình thường'

    df['RSI_Label'] = df['RSI_14'].apply(classify)
    return df

df4 = cal_rsi_14(df3, 14)
df4[['Ngay','MaCK','GiaDongCua','RSI_14','RSI_Label']].head()

/tmp/ipykernel_1327608/885802265.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rsi_series = df.groupby('MaCK').apply(_rsi)


,Ngay,MaCK,GiaDongCua,RSI_14,RSI_Label
2737,2024-01-01,FPT,107300.0,NaN,Bình thường
2978,2024-01-02,FPT,113100.0,NaN,Bình thường
990,2024-01-03,FPT,109600.0,95.564005,Quá mua
555,2024-01-04,FPT,108600.0,94.277195,Quá mua
152,2024-01-05,FPT,110400.0,94.422773,Quá mua


## 8. Xu hướng (2 điểm)
- Nếu Giá > MA5 > MA20 → "Tăng mạnh"
- Nếu Giá > MA20 → "Tăng"
- Nếu Giá < MA5 < MA20 → "Giảm mạnh"
- Nếu Giá < MA20 → "Giảm"
- Còn lại → "Sideway"

In [126]:
df5 = df4.copy()
df5.head(10)

,Ngay,MaCK,GiaMoCua,GiaCaoNhat,GiaThapNhat,GiaDongCua,KhoiLuongGD,GiaTriGD,SoLuongNamGiu,MA5,MA20,MA50,RSI_14,RSI_Label
2737,2024-01-01,FPT,109600.0,113100.0,106000.0,107300.0,4723048.0,5.067831e+11,25000.0,107300.0,107300.000000,107300.000000,NaN,Bình thường
2978,2024-01-02,FPT,111000.0,116400.0,105600.0,113100.0,7924409.0,8.962507e+11,25000.0,110200.0,110200.000000,110200.000000,NaN,Bình thường
990,2024-01-03,FPT,107900.0,109600.0,106200.0,109600.0,7297937.0,7.998539e+11,25000.0,110000.0,110000.000000,110000.000000,95.564005,Quá mua
555,2024-01-04,FPT,109500.0,110800.0,108200.0,108600.0,5968701.0,6.482009e+11,25000.0,109650.0,109650.000000,109650.000000,94.277195,Quá mua
152,2024-01-05,FPT,109600.0,112800.0,106500.0,110400.0,8055860.0,8.893669e+11,25000.0,109800.0,109800.000000,109800.000000,94.422773,Quá mua
3526,2024-01-06,FPT,109500.0,112200.0,106900.0,109000.0,9367255.0,1.021031e+12,25000.0,110140.0,109666.666667,109666.666667,92.452860,Quá mua
3413,2024-01-07,FPT,111600.0,116600.0,106600.0,112000.0,1049663.0,1.175623e+11,25000.0,109920.0,110000.000000,110000.000000,92.799524,Quá mua
2722,2024-01-08,FPT,110200.0,111700.0,108700.0,109700.0,9300264.0,1.020239e+12,25000.0,109940.0,109962.500000,109962.500000,89.408754,Quá mua
1223,2024-01-09,FPT,111800.0,117100.0,106500.0,115500.0,6308395.0,7.286196e+11,25000.0,111320.0,110577.777778,110577.777778,90.364839,Quá mua
2166,2024-01-10,FPT,111900.0,116600.0,107200.0,116100.0,7402161.0,8.593909e+11,25000.0,112460.0,111130.000000,111130.000000,90.460773,Quá mua


In [127]:
def create_xu_huong(row):
    """
    - Nếu Giá > MA5 > MA20 → "Tăng mạnh"
    - Nếu Giá > MA20 → "Tăng"
    - Nếu Giá < MA5 < MA20 → "Giảm mạnh"
    - Nếu Giá < MA20 → "Giảm"
    - Còn lại → "Sideway"
    """
    if pd.isna(row['MA5']) or pd.isna(row['MA20']):
        return 0
    if row['GiaDongCua'] > row['MA5'] > row['MA20']:
        return "Tăng mạnh"
    elif row['GiaDongCua'] > row['MA20']:
        return "Tăng"
    elif row['GiaDongCua'] < row['MA5'] < row['MA20']:
        return "Giảm mạnh"
    elif row['GiaDongCua'] < row['MA20']:
        return "Giảm"
    else:
        return "Sideway"
df5['XuHuong'] = df5.apply(create_xu_huong, axis=1)
df5[['Ngay','MaCK','GiaDongCua','MA5','MA20','XuHuong']].head(20)

,Ngay,MaCK,GiaDongCua,MA5,MA20,XuHuong
2737,2024-01-01,FPT,107300.0,107300.0,107300.000000,Sideway
2978,2024-01-02,FPT,113100.0,110200.0,110200.000000,Tăng
990,2024-01-03,FPT,109600.0,110000.0,110000.000000,Giảm
555,2024-01-04,FPT,108600.0,109650.0,109650.000000,Giảm
152,2024-01-05,FPT,110400.0,109800.0,109800.000000,Tăng
3526,2024-01-06,FPT,109000.0,110140.0,109666.666667,Giảm
3413,2024-01-07,FPT,112000.0,109920.0,110000.000000,Tăng
2722,2024-01-08,FPT,109700.0,109940.0,109962.500000,Giảm mạnh
1223,2024-01-09,FPT,115500.0,111320.0,110577.777778,Tăng mạnh
2166,2024-01-10,FPT,116100.0,112460.0,111130.000000,Tăng mạnh


# Phần 3: Phân tích đầu tư (10 điểm)

## 9. Hiệu suất mã CK (3 điểm)

- Giá đầu kỳ, giá cuối kỳ
- % thay đổi
- Giá cao nhất, thấp nhất trong kỳ
- Volatility (độ lệch chuẩn)

In [120]:
df6 = df5.copy()
def tinh_hieu_suat_mack(data):
    """
    - Giá đầu kỳ, giá cuối kỳ (từ GiaDongCua)
    - % thay đổi
    - Giá cao nhất, thấp nhất trong kỳ
    - Volatility = std của returns (pct_change của GiaDongCua)
    """
    # Đảm bảo dữ liệu sắp xếp theo MaCK và Ngay
    data = data.sort_values(['MaCK', 'Ngay']).reset_index(drop=True)
    # Tính returns hàng ngày theo từng MaCK
    data['ret'] = data.groupby('MaCK')['GiaDongCua'].pct_change()
    # Tổng hợp: dùng GiaDongCua để lấy giá đầu/cuối và max/min; dùng ret để tính volatility
    thong_ke = data.groupby('MaCK').agg({
        'GiaDongCua': ['first', 'last', 'max', 'min'],
        'ret': 'std'
    })
    # Flatten multiindex columns
    thong_ke.columns = ['GiaDauKy', 'GiaCuoiKy', 'GiaCaoNhatKy', 'GiaThapNhatKy', 'Volatility']
    # Tỷ lệ thay đổi và format
    thong_ke['TyLeThayDoi'] = ((thong_ke['GiaCuoiKy'] - thong_ke['GiaDauKy']) / thong_ke['GiaDauKy'] * 100).round(3).astype(str) + '%'
    thong_ke['Volatility'] = thong_ke['Volatility'].round(6)
    thong_ke = thong_ke.reset_index()
    return thong_ke
df6 = tinh_hieu_suat_mack(df6)
df6.head(20)

,MaCK,GiaDauKy,GiaCuoiKy,GiaCaoNhatKy,GiaThapNhatKy,Volatility,TyLeThayDoi
0,FPT,107300.0,100300.0,127500.0,0.0,NaN,-6.524%
1,GAS,121500.0,114200.0,138400.0,0.0,NaN,-6.008%
2,HPG,35900.0,23700.0,36200.0,0.0,NaN,-33.983%
3,MSN,85800.0,79200.0,107700.0,0.0,NaN,-7.692%
4,PLX,54300.0,52400.0,70500.0,0.0,NaN,-3.499%
5,SAB,148300.0,139000.0,172700.0,0.0,NaN,-6.271%
6,VCB,98200.0,91900.0,109800.0,0.0,NaN,-6.415%
7,VHM,65700.0,65200.0,84700.0,0.0,NaN,-0.761%
8,VIC,42900.0,44100.0,60000.0,100.0,35.656478,2.797%
9,VNM,79500.0,76300.0,101100.0,100.0,68.747647,-4.025%


## 10. Phân tích danh mục (4 điểm)

- Giá trị danh mục mỗi ngày = Σ(GiaDongCua × SoLuongNamGiu)
- % biến động giá trị danh mục
- Lãi/Lỗ chưa thực hiện
- Tỷ trọng từng mã trong danh mục

In [121]:
df7 = df5.copy()

In [122]:
def phan_tich_danh_muc(data):
    """
    - Giá trị danh mục mỗi ngày = Σ(GiaDongCua × SoLuongNamGiu) (tổng theo NGÀY)
    - % biến động giá trị danh mục (theo tổng danh mục hàng ngày)
    - Lãi/Lỗ chưa thực hiện (per mã so với giá đầu kỳ của mã đó)
    - Tỷ trọng từng mã trong danh mục (ứng với cùng ngày)
    """
    data = data.copy()
    data['GiaTriDanhMuc'] = data['GiaDongCua'] * data['SoLuongNamGiu']
    # Tổng danh mục theo NGÀY (tổng across MaCK cho mỗi ngày)
    daily_total = data.groupby('Ngay')['GiaTriDanhMuc'].sum().rename('GiaTriDanhMuc_Tong')
    data = data.merge(daily_total.reset_index(), on='Ngay', how='left')
    # Biến động tổng danh mục theo ngày (tính trên tổng theo ngày)
    daily_total_df = daily_total.reset_index().sort_values('Ngay')
    daily_total_df['BienDongGiaTriDanhMuc_Tong'] = daily_total_df['GiaTriDanhMuc_Tong'].pct_change() * 100
    # Gộp lại biến động tổng vào từng hàng tương ứng ngày đó
    data = data.merge(daily_total_df[['Ngay','BienDongGiaTriDanhMuc_Tong']], on='Ngay', how='left')
    # Lãi/Lỗ chưa thực hiện cho mỗi mã: current value - giá trị đầu kỳ của mã đó (first GiaTriDanhMuc mỗi MaCK)
    first_val = data.groupby('MaCK')['GiaTriDanhMuc'].transform('first')
    data['LaiLoChuaThucHien'] = data['GiaTriDanhMuc'] - first_val
    # Tỷ trọng mỗi mã trong tổng ngày
    data['TyTrong'] = data['GiaTriDanhMuc'] / data['GiaTriDanhMuc_Tong'] * 100
    return data
try:
    df7 = phan_tich_danh_muc(df7)
except Exception as e:
    print(f"Loi Ham phan_tich_danh_muc: {e}")
df7.head(10)

,Ngay,MaCK,GiaMoCua,GiaCaoNhat,GiaThapNhat,GiaDongCua,KhoiLuongGD,GiaTriGD,SoLuongNamGiu,MA5,MA20,MA50,RSI_14,RSI_Label,XuHuong,GiaTriDanhMuc,GiaTriDanhMuc_Tong,BienDongGiaTriDanhMuc_Tong,LaiLoChuaThucHien,TyTrong
0,2024-01-01,FPT,109600.0,113100.0,106000.0,107300.0,4723048.0,5.067831e+11,25000.0,107300.0,107300.000000,107300.000000,NaN,Bình thường,Sideway,2.682500e+09,2.849650e+10,NaN,0.0,9.413437
1,2024-01-02,FPT,111000.0,116400.0,105600.0,113100.0,7924409.0,8.962507e+11,25000.0,110200.0,110200.000000,110200.000000,NaN,Bình thường,Tăng,2.827500e+09,2.703150e+10,-5.140982,145000000.0,10.460019
2,2024-01-03,FPT,107900.0,109600.0,106200.0,109600.0,7297937.0,7.998539e+11,25000.0,110000.0,110000.000000,110000.000000,95.564005,Quá mua,Giảm,2.740000e+09,2.772050e+10,2.548878,57500000.0,9.884382
3,2024-01-04,FPT,109500.0,110800.0,108200.0,108600.0,5968701.0,6.482009e+11,25000.0,109650.0,109650.000000,109650.000000,94.277195,Quá mua,Giảm,2.715000e+09,2.763200e+10,-0.319258,32500000.0,9.825565
4,2024-01-05,FPT,109600.0,112800.0,106500.0,110400.0,8055860.0,8.893669e+11,25000.0,109800.0,109800.000000,109800.000000,94.422773,Quá mua,Tăng,2.760000e+09,2.796750e+10,1.214172,77500000.0,9.868597
5,2024-01-06,FPT,109500.0,112200.0,106900.0,109000.0,9367255.0,1.021031e+12,25000.0,110140.0,109666.666667,109666.666667,92.452860,Quá mua,Giảm,2.725000e+09,2.778300e+10,-0.659694,42500000.0,9.808156
6,2024-01-07,FPT,111600.0,116600.0,106600.0,112000.0,1049663.0,1.175623e+11,25000.0,109920.0,110000.000000,110000.000000,92.799524,Quá mua,Tăng,2.800000e+09,2.813600e+10,1.270561,117500000.0,9.951663
7,2024-01-08,FPT,110200.0,111700.0,108700.0,109700.0,9300264.0,1.020239e+12,25000.0,109940.0,109962.500000,109962.500000,89.408754,Quá mua,Giảm mạnh,2.742500e+09,2.849750e+10,1.284831,60000000.0,9.623651
8,2024-01-09,FPT,111800.0,117100.0,106500.0,115500.0,6308395.0,7.286196e+11,25000.0,111320.0,110577.777778,110577.777778,90.364839,Quá mua,Tăng mạnh,2.887500e+09,2.566600e+10,-9.935959,205000000.0,11.250292
9,2024-01-10,FPT,111900.0,116600.0,107200.0,116100.0,7402161.0,8.593909e+11,25000.0,112460.0,111130.000000,111130.000000,90.460773,Quá mua,Tăng mạnh,2.902500e+09,2.362900e+10,-7.936570,220000000.0,12.283635


## 11. Top performers (3 điểm)

- Top 5 phiên tăng mạnh nhất (tất cả mã)
- Top 5 phiên giảm mạnh nhất
- Mã CK có hiệu suất tốt nhất/tệ nhất

In [135]:
df8 = df7.copy()

In [133]:
def top_performers(data, top_5_tang=False, top_5_giam=False, ma_ck_tot_nhat=False, ma_ck_te_nhat=False):
    # Top 5 phiên tăng/giảm mạnh nhất theo tổng danh mục (mỗi ngày duy nhất)
    daily = data[['Ngay', 'GiaTriDanhMuc_Tong', 'BienDongGiaTriDanhMuc_Tong']].drop_duplicates(subset=['Ngay']).sort_values('BienDongGiaTriDanhMuc_Tong', ascending=False)
    
    # Kiểm tra top_5_tang, chỉ thực hiện nếu top_5_tang là True
    if top_5_tang:
        top_5_tang_result = daily.head(5)
        print("Top 5 phiên tăng mạnh nhất (theo tổng danh mục):")
        print(top_5_tang_result)

    # Kiểm tra top_5_giam, chỉ thực hiện nếu top_5_giam là True
    if top_5_giam:
        daily_desc = daily.sort_values('BienDongGiaTriDanhMuc_Tong', ascending=True)
        top_5_giam_result = daily_desc.head(5)
        print("\nTop 5 phiên giảm mạnh nhất (theo tổng danh mục):")
        print(top_5_giam_result)

    # Mã CK có hiệu suất tốt nhất/tệ nhất dựa trên TyLeThayDoi từ tinh_hieu_suat_mack
    hieu_suat = tinh_hieu_suat_mack(data)  # Giả sử tinh_hieu_suat_mack trả về DataFrame chứa cột 'TyLeThayDoi'
    
    if 'TyLeThayDoi' in hieu_suat.columns:
        # Chuyển 'xx.x%' -> float để sắp xếp đúng
        hieu_suat['TyLeThayDoi_num'] = hieu_suat['TyLeThayDoi'].astype(str).str.rstrip('%').astype(float)

        if ma_ck_tot_nhat:
            ma_ck_tot_nhat_result = hieu_suat.sort_values(by='TyLeThayDoi_num', ascending=False).head(1)
            print("\nMã CK có hiệu suất tốt nhất:")
            print(ma_ck_tot_nhat_result[['MaCK', 'TyLeThayDoi']])

        if ma_ck_te_nhat:
            ma_ck_te_nhat_result = hieu_suat.sort_values(by='TyLeThayDoi_num', ascending=True).head(1)
            print("\nMã CK có hiệu suất tệ nhất:")
            print(ma_ck_te_nhat_result[['MaCK', 'TyLeThayDoi']])
    else:
        print('\nKhông tìm thấy cột TyLeThayDoi trong kết quả tinh_hieu_suat_mack')

    # Trả về kết quả để dùng lập trình nếu cần
    result = {}
    if top_5_tang:
        result['top_5_tang'] = top_5_tang_result
    if top_5_giam:
        result['top_5_giam'] = top_5_giam_result
    if ma_ck_tot_nhat:
        result['best'] = ma_ck_tot_nhat_result
    if ma_ck_te_nhat:
        result['worst'] = ma_ck_te_nhat_result

    return result

# Gọi hàm và lưu kết quả vào biến stats (không ghi đè df8)
top_5_tang_data = top_performers(df8, top_5_tang=True)
top_5_giam_data = top_performers(df8, top_5_giam=True)
ma_ck_tot_nhat_data = top_performers(df8, ma_ck_tot_nhat=True)
ma_ck_te_nhat_data = top_performers(df8, ma_ck_te_nhat=True)

Top 5 phiên tăng mạnh nhất (theo tổng danh mục):
          Ngay  GiaTriDanhMuc_Tong  BienDongGiaTriDanhMuc_Tong
98  2024-04-11        2.658750e+10                   38.965112
19  2024-01-20        2.955550e+10                   36.062517
114 2024-04-27        2.567950e+10                   33.949716
235 2024-08-31        3.012350e+10                   31.831510
316 2024-11-21        2.464400e+10                   25.440293

Top 5 phiên giảm mạnh nhất (theo tổng danh mục):
          Ngay  GiaTriDanhMuc_Tong  BienDongGiaTriDanhMuc_Tong
454 2024-04-10        1.913250e+10                  -29.659926
280 2024-10-16        2.008800e+10                  -25.623415
18  2024-01-19        2.172200e+10                  -24.413668
113 2024-04-26        1.917100e+10                  -24.159348
315 2024-11-20        1.964600e+10                  -20.962324

Mã CK có hiệu suất tốt nhất:
  MaCK TyLeThayDoi
8  VIC      2.797%

Mã CK có hiệu suất tệ nhất:
  MaCK TyLeThayDoi
2  HPG    -33.983%


In [138]:
# Dictionary như bạn đã cung cấp
data_dict = {
    'top_5_tang': pd.DataFrame({
        'Ngay': ['2024-04-11', '2024-01-20', '2024-04-27', '2024-08-31', '2024-11-21'],
        'GiaTriDanhMuc_Tong': [2.658750e+10, 2.955550e+10, 2.567950e+10, 3.012350e+10, 2.464400e+10],
        'BienDongGiaTriDanhMuc_Tong': [38.965112, 36.062517, 33.949716, 31.831510, 25.440293]
    })
}

# Chuyển 'top_5_tang' thành DataFrame
top_5_tang_df = data_dict['top_5_tang']

In [139]:
df8.columns

Index(['Ngay', 'MaCK', 'GiaMoCua', 'GiaCaoNhat', 'GiaThapNhat', 'GiaDongCua',
       'KhoiLuongGD', 'GiaTriGD', 'SoLuongNamGiu', 'MA5', 'MA20', 'MA50',
       'RSI_14', 'RSI_Label', 'XuHuong', 'GiaTriDanhMuc', 'GiaTriDanhMuc_Tong',
       'BienDongGiaTriDanhMuc_Tong', 'LaiLoChuaThucHien', 'TyTrong'],
      dtype='object')

In [142]:
df9 = df8[['Ngay', 'MaCK', 'GiaTriDanhMuc', 'GiaTriDanhMuc_Tong', 'TyTrong']].copy()
df9['TyTrong'] = df9['TyTrong'].round(2).astype(str) + '%'
df9.head()

,Ngay,MaCK,GiaTriDanhMuc,GiaTriDanhMuc_Tong,TyTrong
0,2024-01-01,FPT,2.682500e+09,2.849650e+10,9.41%
1,2024-01-02,FPT,2.827500e+09,2.703150e+10,10.46%
2,2024-01-03,FPT,2.740000e+09,2.772050e+10,9.88%
3,2024-01-04,FPT,2.715000e+09,2.763200e+10,9.83%
4,2024-01-05,FPT,2.760000e+09,2.796750e+10,9.87%


In [149]:
with pd.ExcelWriter('output/portfolio_analysis.xlsx') as writer:
    df2.to_excel(writer, sheet_name='DuLieuGiaoDich', index=False)
    df5.to_excel(writer, sheet_name='ChiSoKyThuat', index=False)
    df6.to_excel(writer, sheet_name='HieuSuatMaCK', index=False)
    df7.to_excel(writer, sheet_name='GiaTriDanhMuc', index=False)
    top_5_tang_df.to_excel(writer, sheet_name='TopPhien', index=False)
    df9.to_excel(writer, sheet_name='TyTrongDanhMuc', index=False)
    bao_cao_tong_hop.to_excel(writer, sheet_name='BaoCaoTongHop', index=False)

print('Done!')

## Bao cao Tong Hop

In [148]:
bao_cao_tong_hop = df8.copy()
bao_cao_tong_hop['TyTrong'] = bao_cao_tong_hop['TyTrong'].round(2).astype(str) + '%'
def chuan_hoa_BienDongGiaTriDanhMuc_Tong(row):
    if pd.isna(row):
        return '0%' 
    else:
        return str(round(row, 3)) + '%' 
bao_cao_tong_hop['BienDongGiaTriDanhMuc_Tong'] = bao_cao_tong_hop['BienDongGiaTriDanhMuc_Tong'].apply(chuan_hoa_BienDongGiaTriDanhMuc_Tong)

print(bao_cao_tong_hop.head())


        Ngay MaCK  GiaMoCua  GiaCaoNhat  GiaThapNhat  GiaDongCua  KhoiLuongGD  \
0 2024-01-01  FPT  109600.0    113100.0     106000.0    107300.0    4723048.0   
1 2024-01-02  FPT  111000.0    116400.0     105600.0    113100.0    7924409.0   
2 2024-01-03  FPT  107900.0    109600.0     106200.0    109600.0    7297937.0   
3 2024-01-04  FPT  109500.0    110800.0     108200.0    108600.0    5968701.0   
4 2024-01-05  FPT  109600.0    112800.0     106500.0    110400.0    8055860.0   

       GiaTriGD  SoLuongNamGiu       MA5      MA20      MA50     RSI_14  \
0  5.067831e+11        25000.0  107300.0  107300.0  107300.0        NaN   
1  8.962507e+11        25000.0  110200.0  110200.0  110200.0        NaN   
2  7.998539e+11        25000.0  110000.0  110000.0  110000.0  95.564005   
3  6.482009e+11        25000.0  109650.0  109650.0  109650.0  94.277195   
4  8.893669e+11        25000.0  109800.0  109800.0  109800.0  94.422773   

     RSI_Label  XuHuong  GiaTriDanhMuc  GiaTriDanhMuc_Tong  \
